Server allocation problem
=====================

You have $n$ virtual machines, each of them requires some amount of cpu, ram and network throughput. You want to buy as little servers as possible to run them. Each server is built on the same template.

In [1]:
# Let's follow Maria's advice and use namedtuples
from collections import namedtuple
Machine = namedtuple('Machine', ['ram', 'cpu', 'net'])

In [2]:
# An instance of the problem

# server template
server = Machine(ram=256, cpu=32, net=100)

# possible vms
small = Machine(ram=8, cpu=1, net=10)
medium = Machine(ram=16, cpu=4, net=15)
big = Machine (ram=128, cpu=8, net=40)

# An instance of our problem
instance = { 'server' : server, 'vms': 10 * [ small ] + 5 * [ medium ] + 2 * [ big ] }

In [3]:
# Helper function
def well_defined(instance):
    server = instance["server"]
    vms = instance["vms"]
    for vm in vms:
        if vm.cpu > server.cpu or vm.ram > server.ram or vm.net > server.net:
            return False
    return True

print(well_defined(instance))
print(well_defined({"server": Machine(1, 1 ,1), "vms": [Machine(2, 2, 2)]}))

True
False


A solution is to greedily assign vms to the same server as long as they fit, and switch to the next server otherwise. Let's talk about a linear programming model that yields the optimal solution instead.

Model
-----

* Let $x_{i, j}$ be the variable "the vm $i$ is assigned to the server $j$". That is, if $x_{i, j} = 1$, the vm is assigned to the server, and otherwise $x_{i, j} = 1$.
* Let $y_j$ be the variable "the server $j$ is used".

Initially we are going to assume that there are $n$ servers (as many as the vms), because it is a feasible solution. It allows to have a bounded number of $y_j$.

Let $M$ be a large number. A "trick" to express the fact that, for a given server $j$, the fact any of $x_{i, j}=1$ means that the server is used, is to use the following constraint: $\sum_{i=1}^n x_{i, j} <= M y_j$.

The model can be expressed like this:

* minimize $\sum_{j=1}^n y_j$ (we want to minimize the number of servers)
* such that
* $\forall i \in [1..n], \sum_{j=1}^n x_{i, j} = 1$ ( = each vm $i$ is assigned to exactly one server)
* $\forall j \in [1..n], \sum_{i=1}^n x_{i, j} <= M y_j$ (if any of the vms uses the server $j$, its associated variable cannot be null)
* $\forall j \in [1..n], \sum_{i=1}^n x_{i, j} \text{cpu}_i <= \text{cpu}$ (all vm must fit into the server in terms of cpu)
* $\forall j \in [1..n], \sum_{i=1}^n x_{i, j} \text{ram}_i <= \text{ram}$ (all vm must fit into the server in terms of ram)
* $\forall j \in [1..n], \sum_{i=1}^n x_{i, j} \text{net}_i <= \text{net}$ (all vm must fit into the server in terms of network throughput)

All right, but there is far too much symetry because are solvers are the same, so a linear solver will spend a lot of time looking for the optimal solution because they are virtually all worth the same. A way to break the symetry is to introduce an increasing cost for each new server. This way the server $n$ is going to cost more than the server $n-1$, which is going to cost more than the server $n-2$... So instead of minimizing $\sum_{j=1}^n y_j$, we minimize e.g. $\sum_{j=1}^n j y_j$.

In [4]:
from pulp import *

def solve(instance):
    if not well_defined(instance):
        raise ValueError("Infeasible problem")
    
    vms = instance["vms"]
    server = instance["server"]
    n = len(vms)
    
    # variables
    x = {(i, j): LpVariable("X_%s_%s" % (i, j), 0, 1, LpInteger) for i in range(n) for j in range(n)}
    y = [ LpVariable("Y_srv%s" % j, 0, 1, LpInteger) for j in range(n) ]
    
    # problem
    prob = LpProblem("Servers allocation", LpMinimize)
    
    # objective
    costs = range(1, n+1)
    prob += lpSum([ c * y for y, c in zip(costs, y)])
    
    # each vm assigned exactly once
    for i in range(n):
        prob += lpSum([x[(i, j)] for j in range(n)]) == 1, "vm %s assigned once" % i
    
    # sum_i xij >= 1 => server j in use
    for j in range(n):
        prob += lpSum([x[(i, j)] for i in range(n)]) <= n * y[j]
        
    # the assigned cpu fits
    for j in range(n):
        prob += lpSum([x[(i, j)] * vms[i].cpu for i in range(n)]) <= server.cpu
    
    # the assigned ram fits
    for j in range(n):
        prob += lpSum([x[(i, j)] * vms[i].ram for i in range(n)]) <= server.ram
        
    # the assigned net fits
    for j in range(n):
        prob += lpSum([x[(i, j)] * vms[i].net for i in range(n)]) <= server.net
    
    # Let the solver do the magic
    prob.solve()
    
    res = {}
    for i in range(n):
        for j in range(n):
            if x[(i,j)].varValue == 1:
                res.setdefault(j, []).append(i)
    return res

    

In [5]:
server_to_vms = solve(instance)
server_to_vms

{0: [0, 2, 3, 4, 5, 6, 7, 8, 9], 2: [1, 12, 16], 1: [10, 11, 13, 14, 15]}